In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from statsmodels.nonparametric.smoothers_lowess import lowess
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.preprocessing import normalize, scale, Normalizer, StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR, LinearSVC
from sklearn.feature_extraction.text import CountVectorizer

import numpy as np
import pandas as pd
import pickle 
import os, re
from PIL import Image
import pathlib
import csv
import urllib

from sklearn import datasets

# pip install git+git://github.com/mgelbart/plot-classifier.git
from plot_classifier import plot_classifier

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
pd.set_option("display.max_colwidth", 200)

import altair as alt

# pip install ipython-autotime
import autotime

In [2]:
# Read IMDB movie reviews into a pandas DataFrame
imdb_df = pd.read_csv('data/imdb_master.csv', encoding = "ISO-8859-1")
# Only keep the reviews with pos and neg labels
imdb_df = imdb_df[imdb_df['label'].str.startswith(('pos','neg'))]
imdb_df.head(4)

,Unnamed: 0,type,review,label,file
0,0,test,"Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the charact...",neg,0_2.txt
1,1,test,"This is an example of why the majority of action films are the same. Generic and boring, there's really nothing worth watching here. A complete waste of the then barely-tapped talents of Ice-T and...",neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who could'nt act if they had a gun pressed against their foreheads. All they do is curse and shoot each other and acting like clichÃ©'e version of gangst...",neg,10001_1.txt
3,3,test,"Not even the Beatles could write songs everyone liked, and although Walter Hill is no mop-top he's second to none when it comes to thought provoking action movies. The nineties came and social pla...",neg,10002_3.txt


In [3]:
X_train, X_valid, y_train, y_valid = train_test_split(imdb_df['review'], 
                                                                        imdb_df['label'], 
                                                                        test_size = 0.20, 
                                                                        random_state = 12)
vec = CountVectorizer(max_features=5000,                       
                      stop_words='english')

X_train_counts = vec.fit_transform(X_train) 
  
X_test_counts = vec.transform(X_valid)

In [4]:
lr = LogisticRegression()
lr.fit(X_train_counts, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [5]:
vocab = vec.get_feature_names()
weights = lr.coef_.flatten()

inds = np.argsort(lr.coef_.flatten())

negative_words = [vocab[index] for index in inds[:10]]

neg_words_weights = [(weights[index]) for index in inds[:10]]

results = pd.DataFrame({'Neg Words':negative_words, 'Weights':neg_words_weights})

results.to_csv(r'table\negative.csv')

In [6]:
results

,Neg Words,Weights
0,waste,-2.250113
1,disappointment,-2.175797
2,mst3k,-2.171390
3,mildly,-1.952146
4,forgettable,-1.908331
5,laughable,-1.896478
6,uninspired,-1.881927
7,lousy,-1.820625
8,redeeming,-1.773734
9,worst,-1.749149
